# Grep demo

In [3]:
import iknowpy

iknow = iknowpy.iKnowEngine()

In [55]:
iknow.index("The patient did not report kidney beans, but had a lower back pain after gardening on Saturday.","en")

In [67]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(iknow.m_index)

{ 'proximity': [ ((5, 7), 64),
                 ((1, 3), 64),
                 ((3, 5), 64),
                 ((3, 7), 42),
                 ((1, 5), 42),
                 ((1, 7), 32)],
  'sentences': [ { 'entities': [ { 'dominance_value': 0.0,
                                   'entity_id': 0,
                                   'index': 'the',
                                   'offset_start': 0,
                                   'offset_stop': 3,
                                   'type': 'NonRelevant'},
                                 { 'dominance_value': 333.0,
                                   'entity_id': 1,
                                   'index': 'patient',
                                   'offset_start': 4,
                                   'offset_stop': 11,
                                   'type': 'Concept'},
                                 { 'dominance_value': 1000.0,
                                   'entity_id': 2,
                                   'index':

In [57]:
# print basic parsing output
for s in iknow.m_index['sentences']:
    for e in s['entities']:
        print('<'+e['type']+'>'+e['index']+'</'+e['type']+'>', end=' ')
    print('\n')

<NonRelevant>the</NonRelevant> <Concept>patient</Concept> <Relation>did not report</Relation> <Concept>kidney beans</Concept> <Relation>but had</Relation> <NonRelevant>a</NonRelevant> <Concept>lower back pain</Concept> <Relation>after gardening on</Relation> <Concept>saturday</Concept> 



In [59]:
# print negation output
for s in iknow.m_index['sentences']:
    
    # first build an array marking start and end positions
    pos = 0
    negated = []
    for a in s['path_attributes']:
        if a['type']=="Negation":
            print(a)
            for p in range(pos,a['pos']-1):
                negated.append(0)
            for p in range(a['pos'],a['pos']+a['span']):
                negated.append(1)
            pos = a['pos']+a['span']
    for p in range(pos, len(s['entities'])):
        negated.append(0)
    print(negated)
    
    pos = 0
    for e in s['entities']:
        if e['type'] == 'NonRelevant': continue
        if negated[pos]: print("<neg>", end='')
        print(e['index'], end='')
        if negated[pos]: print("</neg>", end='')
        print(" ", end="")
        pos = pos+1
    print('\n')

{'type': 'Negation', 'pos': 0, 'span': 3}
[1, 1, 1, 0, 0, 0, 0, 0, 0]
<neg>patient</neg> <neg>did not report</neg> <neg>kidney beans</neg> but had lower back pain after gardening on saturday 



In [60]:
# now turn into a function
def strip_negative(text, language="en", iknow=iknowpy.iKnowEngine()):
    iknow.index(text,language)
    stripped = ""
    for s in iknow.m_index['sentences']:
        pos = 0
        negated = []
        for a in s['path_attributes']:
            if a['type']=="Negation":
                #print(a)
                for p in range(pos,a['pos']-1):
                    negated.append(0)
                for p in range(a['pos'],a['pos']+a['span']):
                    negated.append(1)
                pos = a['pos']+a['span']
        for p in range(pos, len(s['entities'])):
            negated.append(0)
        pos = 0
        for e in s['entities']:
            if e['type'] == 'NonRelevant': continue
            if negated[pos] != 1: 
                stripped += e['index']
                stripped+= " "
            pos = pos+1
        stripped += '\n'
    return stripped
        

print(strip_negative("he wasn't very happy with the cake, even though we put much effort (and no sugar) into it."))

even though we put much effort no sugar into it 

